## rLab Door Controller Design





### Introduction

The rLab door failed meaning that members could not access rLab using their RFID cards.  It turned out to be an issues with the actuator in the door, but whilst troubleshooting it became apparent that the control circuitry wasn't in a great state.  I have offered to create a tidyier replacement; this is the design document.

#### Current System

![What a mess](img/20220615_Electronics_Overview.jpg)

The current system comprises:

* Raspberry Pi - Provides overall control, allows network access, syncs members data from the members database, and authenticates RFID card access
* Arduino - interfaces between the Pi and the 12V switching needed to operate the door
* Power Supply Board - Converts incoming DC to voltages used around the system, including battery charging for a sealed lead acid battery which ensures the door continues to operate during a power cut
* Relay Interface Board - Currently sits as a shield above the Arduino providing connectivity to the relay which provides a 12V pulse to open the door.

### Scope of the Project


#### Out of Scope

* The Raspberry Pi functionality is outside the scope of this project

#### In Scope

* Power supplies including to the Pi
* Battery backup and charging
* Control of the door actuator under the command of the Pi
* Add a display and/or LEDs to help troubleshooting
* Change connections to plug and socket or screw terminals, instead of random twisted wires

### Design Criteria

#### Design Principles

* Aim for simplicity
* Without breaking the first principle, enable flexibility of the design
* Enable easy troubleshooting
  * Provide displays or indications of fault conditions
  * Provide serial log output

### Power Supply


#### Requirements

1. Must operate the door actuator which is quoted as 2.5A @ 12V
2. Must provide power to a Raspberry Pi, the latest Pi4 states 3A 5V supply, but lower power versions are available.  It may also be possible to power the Pi using a 3.3V supply through the expansion header.
3. Must provide power to the microcontroller, likely 3.3V
4. Must be able to operate the door through a power cut - implies battery backup.


##### Mains Voltage

To make this project as safe as possible there will be no mains voltages used in the design.  DC power input will be provided by an encapsulated external power supply.


##### Power Input

The input power will need to be at least 14V DC to charge backup batteries.  Because of dropout in the charging system it is likely that the input voltage will need to be at least 17V.  Ideal power supply voltage with be 18V-20V DC.  This range is easily achieved with laptop style power supplies.


##### Power Rails

* $V_{In}$ = 18-24V
  * Provided by external encapsulated power supply
* $V_{Batt}$ = 12V nominal (13.8V likely when charging)
  * Needed to drive the door actuator
  * Sealed lead-acid battery provides backup power on this rail
* $+5V$
  * Power for Raspberry Pi
  * Derived from $V_{Batt}$ using an efficient switching regulator, like [this](https://www.amazon.co.uk/dp/B08DQVJZH3)
* $+3.3V$
  * Power for microcontroller
  * Derived from $+5V$ rail using an LDO linear regulator, such as the [AMS1117-3.3](https://octopart.com/datasheet/ams1117-3.3-ams-26604089)


#### Battery Charging

Sealed lead-acid betteries are the standard backup supply used in fire and security alarms.  They are well suited to trickle charging with few discharge cycles.  Lithium cells are less well suited and are not therefore being considered.

To trickle charge sealed lead-acid batteries this [article](https://www.power-sonic.com/blog/how-to-charge-a-lead-acid-battery/) suggests that you need to apply 2.3V/cell.  So for a 12V, 6 cell battery this would be in the region of 13.8V.

> The recommended constant float voltage is 2.25V-2.30V per cell. Maintaining this float voltage will allow the battery to define its own current level and remain fully charged without having to disconnect the charger from the battery. The trickle current for a fully charged battery floating at the recommended charge voltage will typically hover around the 0.001C rate (2mA for a 2Ah battery for example.)


##### High Performance Charger (UC3906)

The [datasheet](https://www.farnell.com/datasheets/45075.pdf) states that this can be configured to manage the charging and performance of lead-acid batteries, but the circuits are more complex and the solution is expensive, around £10/chip, and there seems to be a [lack of supply](https://octopart.com/search?q=uc3906) for these at present.

Although this would provide more detailed information of the charging state over I2C to the microcontroller the availability and cost mean that I am discounting this option.  Also it is probably overkill for the standby trickle charge situation where charging speed is not critical.

##### Simple option (LM317)

The [datasheet](https://www.ti.com/lit/ds/symlink/lm317.pdf) has the following application for constant voltage battery charging:

![LM317 Battery Charging Circuit](img/LM317_Charger.png)

$R_s$ is used to protect the battery from high current or to limit the output in case of shortcircuit.  The application note suggests $R_s = 0.2\Omega$ this would give a short circuit current of around 6A, which would then be limited by the LM317 to around 0.5A-1.5A depending on the model.  The datasheet says the following:

> When an overload occurs the device shuts down Darlington NPN output stage or reduces the output current to prevent device damage. The device will automatically reset from the overload. The output may be reduced or alternate between on and off until the overload is removed.

So the maximum power dissipation needed for $R_s$ is around 0.5W for the 1.5A version and 0.05W for the 500mA version.

$R_1$ and $R_2$ set the output voltage.  With the given values of $R1=240\Omega$ and $R2=2.4k\Omega$ the output voltage is:

$ V_{out} = V_{ref} \left(1 + \frac{R_2}{R_1}\right) $

$ V_{out} = 1.25 \left(1 + \frac{2400}{240}\right) = 13.75V  $

These values are therefore good for the trickle charge application.  However, because of potential tolerances in the components it would be sensible to make $R2 = 3K\Omega$ variable resistor.

These ICs are around £1.50 each and currently seem to be [available](https://octopart.com/lm317p-stmicroelectronics-530526?r=sp) to purchase.


In [8]:
R1 = 240
R2 = 2400
VRef = 1.25
VOut = VRef * (1 + R2/R1)
print(f"The output voltage is: {VOut}V")

The output voltage is: 13.75V


## Design Notes

### Desirable in the PCB Design

* LED and Test points for each rail
* LED and Test Point for Actuator output
* Earth Stud for test point ease of use
* Provide dual foorprint for SMD and through-hole LEDs
* Make $R2$ adjustable so we can fine tune the charge voltage
* 16-Pin Character Display Interface
* USB Programming and serial console
* Neopixel(s)?
* Actuate pin should be capable of driving 2.5A.  This is not required for the current door, as this is a signal only and the door also has a 12V supply but future doors may require direct signal drive.

## USB-C connector

Thos [article](https://pcbartists.com/design/embedded/how-to-replace-microusb-with-usb-c-connector/) shows the following schematic for using a USB-C connector with USB 2.0 protocol.

![USB-C Connector for USB 2.0 Schematic](img/USB-C_2.0.png)
